In [10]:
import h5py
import db
import series


from pprint import pprint


def add_series_to_sql(series_name=None, series_num=None):
    
    ser = Series(series_name=series_name, series_num=series_num)
    
    series_info = ser.get_general_info()
    
    server = db.MySQLCore('pylelab_daq')
    server.connect_test()
    
    server.insert('series', series_info)
    
    server.disconnect()
    
    
   

In [11]:
def add_group_to_sql(group_name, group_path):
    
    group = series.SeriesGroup(group_name, group_path)
    
    print(1)
    
    group_info, series_info = group.get_group_info(include_series_info=True)
    
    print(group_info)
    
    server = db.MySQLCore('pylelab_daq')
    server.connect_test()
    
    print("connected")
    
    server.insert('groups', group_info)
    
    for s in series_info:   
        series_info[s].pop('ADC_config')
        series_info[s].pop('detector_config')
        series_info[s].pop('connection_table')
        
        #print(series_info[s])
        
        server.insert('series', series_info[s])
    
    server.disconnect()

In [3]:
def get_metadata(file_name):
    
    import hdf5
    
    hdf5 = hdf5.H5Core()
    hdf5.set_files(file_name)
    info = hdf5.get_file_info()
    #pprint(info)
    
    nameset = ['id', 'series_name', 'series_num', 'facility', 'fridge_run', 'data_type',
           'data_purpose', 'nb_events', 'nb_dumps', 'timestamp', 'comment']
    keyset = {'data_type': 'run_type', 'data_purpose': 'run_mode' }
    dataset = {}
    
    for name in nameset:
        if name in keyset: 
            n = keyset[name]
        else: n = name
            
        if n in info:  
            dataset[name] = info[n]
            print("insert")
            
        else: dataset[name] = "NULL"
            
            
    dataset['nb_events']=info['adc1']['nb_events']
    sname = file_name.split('/')[-1][:-11]
    dataset['series_name'] = sname
    dataset['series_num'] = int(''.join((ch if ch in '0123456789' else '') for ch in sname))
    
    return dataset

In [17]:
#add_group_to_sql('exttrigger_I2_D20211110_T103901', '/sdata/runs/run12/raw')
add_group_to_sql('iv_didv_I2_D20211104_T084659', '/sdata/runs/run12/raw')
#add_group_to_sql('pton_didv_I2_D20211110_T114610', '/sdata/runs/run12/raw')



1
{'group_name': 'iv_didv_I2_D20211104_T084659', 'facility': 2, 'fridge_run': 12, 'nb_series': 76, 'nb_events': 304361, 'duration': 305.46, 'devices': 'CPDv2-Al2O3', 'data_types': '103, 102', 'data_purposes': 'dIdV, IV', 'group_comment': 'IV/dIdV sweep PT ON', 'timestamp': 1636040843}
Database successfully connected!
connected
Database successfully disconnected.


In [13]:

server = db.MySQLCore('pylelab_daq')
server.connect_test()


series = server.query("groups", values=['group_name'])

server.disconnect()


print(series)

Database successfully connected!
Database successfully disconnected.
[{'group_name': 'exttrigger_I2_D20211110_T103901'}, {'group_name': 'iv_didv_I2_D20211104_T084659'}]


In [4]:
server = db.MySQLCore('pylelab_daq')
server.connect_test()


Database successfully connected!


In [7]:
server.disconnect()

Database successfully disconnected.


In [30]:
#### FINAL LOOP ####

table_name = 'test'
#for file in folder:
fname = "trigger_I2_D20210324_T104549_F0003.hdf5"
dataset = get_metadata(fname)
sname = fname.split('/')[-1][:-11]
timestamp = int(''.join((ch if ch in '0123456789' else '') for ch in sname)[1:])

print(timestamp)

last = int(server.get_last(table_name))

if timestamp >= last: 
    server_query = next((item for item in server.query(table_name, ["series_name"])
                         if item["series_name"] == sname), None)
    if server_query:
        server.modify(table_name, dataset)
        print("mod")
    else:
        server.insert(table_name, dataset)
        print("add")
        
        # for new files: iterate thru all files of a series, compare new number of dumps to old, edit num. dumps and num. events  


insert
insert
insert
20210324104549
mod


In [15]:
grouplist = server.query("groups")
def search(dic):
        return dic['group_name']

grouplist.sort(key=search)
print(grouplist)

[{'group_name': 'testdaq_I2_D20211108_T225300', 'facility': 2, 'fridge_run': 12, 'data_types': '1', 'data_purposes': 'Test', 'duration': 2, 'devices': 'CPDv2-Al2O3', 'timestamp': 1636440784, 'nb_series': 2, 'nb_events': 122, 'group_comment': 'Continuous data PT ON, ~27% Rn, 2 weeks since at base'}, {'group_name': 'testdaq_I2_D20211108_T225300', 'facility': 2, 'fridge_run': 12, 'data_types': '1', 'data_purposes': 'Test', 'duration': 2, 'devices': 'CPDv2-Al2O3', 'timestamp': 1636440784, 'nb_series': 2, 'nb_events': 122, 'group_comment': 'Continuous data PT ON, ~27% Rn, 2 weeks since at base'}, {'group_name': 'testdaq_I2_D20211108_T225300', 'facility': 2, 'fridge_run': 12, 'data_types': '1', 'data_purposes': 'Test', 'duration': 2, 'devices': 'CPDv2-Al2O3', 'timestamp': 1636440784, 'nb_series': 2, 'nb_events': 122, 'group_comment': 'Continuous data PT ON, ~27% Rn, 2 weeks since at base'}, {'group_name': 'testdaq_I2_D20211108_T225300', 'facility': 2, 'fridge_run': 12, 'data_types': '1', 'da

In [3]:
server.set_last('1234')

You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'DECLARE @last INT' at line 1


[]

In [ ]:
### PAST ATTEMPTS ###

 hdf5 = hdf5.H5Core()
    hdf5.set_files(file_name)
    info = hdf5.get_file_info()
    pprint(info)
    
    nameset = ['id', 'series_name', 'series_num', 'facility', 'fridge_run', 'data_type',
           'data_purpose', 'nb_events', 'nb_dumps', 'timestamp', 'comment']
    keyset = {'data_type': 'run_type', 'data_purpose': 'run_mode' }
    dataset = {}
    
    for name in nameset:
        print(name)
        if name in keyset: 
            n = keyset[name]
        else: n = name
            
        if n in info:  
            dataset[name] = info[n]
            print("insert")
            
        else: dataset[name] = "NULL"
            
            
    dataset['nb_events']=info['adc1']['nb_events']
    dataset['series_name'] = file_name.split('/')[-1][:-11]
    
    server = MySQLCore('pylelab_daq')
    server.connect_test()
    
    server.insert('test', dataset)
    print(dataset)
    
    server.disconnect()